# Projet du cours "Kernel methods" du master MVA
Le projet de ce cours consiste en la classification d'images. Dans ce notebook, on implémente de manière la plus détaillée et la plus claire possible l'ensemble des classes et fonctions des algorithmes que nous utilisons.    

Ce projet se fera en 3 étapes :
- Preprocessing des données
- Feature extraction
- Prediction par SVM

Il est indiqué dans l'énoncé que l'étape de preprocessing a déjà été réalisée. Nous allons donc directement procéder à l'extraction de nos features (par HOG) et à la prédiction de la nature de nos images (par SVM).

Ce notebook utilise comme kernel le RBF kernel qui est celui qui donne les meilleurs résultats. Pour voir les résultats avec d'autres kernels, se reporter aux notebooks joints (moins détaillés mais présentant les principaux résultats).

## Chargement des modules

In [1]:
%pylab inline
import pandas
import cvxopt
import time
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt 
import scipy
from scipy.stats import mode
from scipy.linalg import eigh

Populating the interactive namespace from numpy and matplotlib


# Importation des données

On importe les données du concours

In [2]:
df_train=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xtr.csv',header=None,sep=',')
df_train=df_train.drop(3072,1)
df_test=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xte.csv',header=None,sep=',')
df_test=df_test.drop(3072,1)
y_train=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Ytr.csv',sep=';')
y_train=pandas.DataFrame(y_train['Prediction'])

In [3]:
X_train = np.genfromtxt('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xtr.csv',delimiter=',')[:,0:3072]
Y_train = np.genfromtxt('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Ytr.csv',delimiter=';')[1:5001,1]
X_test = np.genfromtxt('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xte.csv',delimiter=',')[:,0:3072]

# Répartition des labels
Nous travaillons sur un échantillon d'apprentissage dans le but de classifier des images. Vérifions que nous avons bien équirépartition des images dans l'échantillon d'apprentissage afin de pouvoir mener à bien notre modélisation

In [4]:
y_train["Prediction"].value_counts()

7    500
3    500
6    500
2    500
9    500
5    500
1    500
8    500
4    500
0    500
Name: Prediction, dtype: int64

# Implémentation des méthodes

Dans cette partie, nous implémenterons les différentes méthodes que nous testerons par la suite. Il s'agit des méthodes suivantes:
- SVM
- ACP
- HOG



## Support Vector Machine

In [5]:
class SVMTrainer(object):
    def __init__(self, c, sigma):
        self._c = c
        self.sigma=sigma
        
    def train(self, X, y):
        lagrange_multipliers = self._compute_multipliers(X, y)
        return self._construct_predictor(X, y, lagrange_multipliers)

    def _gram_matrix(self, X):
        n_samples, n_features = X.shape
        v=norm(X,2,axis=1)**2
        v2=array([v,]*n_samples)+array([v,]*n_samples).transpose()-2*dot(X.values,transpose(X.values))
        K=exp(-v2/(2*(self.sigma)**2)) #utilisation de la définition de la norme deux pour une implémentation simple
        print("Taille de la matrice de Gram :",K.shape)
        return K

    def _construct_predictor(self, X, y, lagrange_multipliers):
        support_vector_indices=abs(lagrange_multipliers)>0
        support_multipliers = lagrange_multipliers[support_vector_indices] # alpha i positifs
        support_vectors = X[support_vector_indices] # vecteurs support
        support_vector_labels = y[support_vector_indices] # labels des vecteurs support
        print("Nombre de vecteurs supports :",len(support_vectors))
        pre=SVMPredictor(sigma=self.sigma,bias=0.0,weights=support_multipliers,
             support_vectors=support_vectors,support_vector_labels=support_vector_labels).predictest(support_vectors.values)[0]
        bias = np.mean(support_vector_labels[:,0]-pre)
        print("Biais:",bias)
        return SVMPredictor(sigma=self.sigma,bias=bias,weights=support_multipliers,support_vectors=support_vectors,
            support_vector_labels=support_vector_labels)

    def _compute_multipliers(self, X, y):
        n_samples, n_features = X.shape
        K=self._gram_matrix(X)
        P=cvxopt.matrix(K)
        q=cvxopt.matrix(-1*y[:,0])
        G_std=cvxopt.matrix(-1*np.diag(y[:,0]))
        h_std=cvxopt.matrix(np.zeros(n_samples))
        G_slack=cvxopt.matrix(np.diag(y[:,0]))
        h_slack=cvxopt.matrix(np.ones(n_samples) * self._c)
        G=cvxopt.matrix(np.vstack((G_std, G_slack)))
        h=cvxopt.matrix(np.vstack((h_std, h_slack)))
        A=cvxopt.matrix(np.ones(n_samples),(1,n_samples))
        b=cvxopt.matrix(0.0)
        solution = cvxopt.solvers.qp(P, q, G, h, A=None, b=None)
        return np.ravel(solution['x'])

In [6]:
class SVMPredictor(object):
    def __init__(self,sigma,bias,weights,support_vectors,support_vector_labels):
        self.sigma=sigma
        self._bias = bias
        self._weights = weights
        self._support_vectors = support_vectors
        self._support_vector_labels = support_vector_labels

    def predictest(self,x):
        v = array([norm(x,2,axis=1)**2,]*len(self._support_vectors))
        inter = array([norm(self._support_vectors,2,axis=1)**2,]*len(x)).transpose() + v
        inter1 = inter - 2*dot(self._support_vectors,transpose(x))
        inter2 = exp(-asarray(inter1)/(2*(self.sigma)**2))
        result = dot(transpose(self._weights),inter2)+self._bias
        return sign(result),result

## Adaptation du SVM au cas multi-classe
On va tout d'abord implémenter la méthode One Vs One, i.e. considérer 10x9/2=45 problèmes de classifications binaires en traitant des paires d'instances, puis implémenter la méthode One Vs Rest, i.e. entraîner un classifieur par classe.

### One Vs One

In [7]:
def SVMOvO(data_train,data_test,is_train,C,sigma):
    SVM=SVMTrainer(C,sigma)
    result=(20*ones(len(data_test))).reshape(len(data_test),1)
    for i in [0,1,2,3,4,5,6,7,8]:
        for j in range(i+1,10):
            print("Classe :",i,j)
            df_train_ij=data_train[asmatrix(is_train,dtype=ndarray)==(i,j)].copy()
            is_train_ij=is_train[asmatrix(is_train,dtype=ndarray)==(i,j)].copy()
            is_train_ij[is_train==i]=1
            is_train_ij[is_train==j]=-1
            s1=time.clock()
            SVMPred=SVM.train(df_train_ij,asfarray(is_train_ij))
            s2=time.clock()
            print("Temps pour l'entrainement :",s2-s1)
            s3=time.clock()
            values=SVMPred.predictest(data_test.values)[0]
            s4=time.clock()
            print("Temps pour la prédiction :",s4-s3)
            values[values==1]=i
            values[values==-1]=j
            values=values.reshape(len(data_test),1)
            result=concatenate((result,values),1)
            is_test_predict=mode(result,axis=1)
    return(is_test_predict)    

### One Vs Rest

In [8]:
def SVMOvR(data_train,data_test,is_train,C,sigma):
    SVM=SVMTrainer(C,sigma)
    result=(-20*ones(len(data_test))).reshape(len(data_test),1)
    for i in [0,1,2,3,4,5,6,7,8,9]:
        print("Classe :",i)
        is_train_i=is_train.copy()
        is_train_i[is_train==i]=1
        is_train_i[is_train!=i]=-1
        s1=time.clock()
        SVMPred=SVM.train(data_train,asfarray(is_train_i))
        s2=time.clock()
        print("Temps pour l'entrainement :",s2-s1)
        s3=time.clock()
        values=SVMPred.predictest(data_test.values)[1]
        s4=time.clock()
        print("Temps pour la prédiction :",s4-s3)
        values=values.reshape(len(data_test),1)
        result=concatenate((result,values),1)
    predict=argmax(result,axis=1)-1
    return(predict)  

# Kernel - Analyse en Composantes Principales



In [9]:
def kpca(X, sigma, n_components):

    # Gram matrix 
    n_samples, n_features = X.shape
    v=norm(X,2,axis=1)**2
    v2=array([v,]*n_samples)+array([v,]*n_samples).transpose()-2*dot(X.values,transpose(X.values))
    K=exp(-v2/(2*sigma**2))
    
    # Center 
    N = K.shape[0]
    one_n = np.ones((N,N)) / N
    K = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)

    # Eigenvalues in descending order with corresponding eigenvectors 
    eigvals, eigvecs = eigh(K)

    # Eigenvectors that corresponds to the highest eigenvalues.
    X_kpca = np.column_stack((eigvecs[:,-i] for i in range(1,n_components+1)))

    return X_kpca

# HOG

Implémentation de la méthode d'extraction de features inspirée de l'article https://hal.inria.fr/inria-00548512/document de N. Dalal et B. Triggs et du site http://www.learnopencv.com/histogram-of-oriented-gradients/. On choisit de diviser l'image de 32 pixels en 4 cellules de (8,8) pixels chacun, en normalisant par blocs de 4 cellules, ce qui donne une base X de taille 3 (nombre de blocs sur l'axe x) fois 3 (nombre de blocs sur l'axe y) fois 9 (taille d'un histogramme) fois 4 (nombre d'histogrammes par bloc, un histogramme correspondant à une cellule) fois 3 (pour les différentes couleurs de l'image), i.e. on obtient une base X avec 972 variables au lieu de 3072.

In [10]:
class HOG :

    def __init__(self, nbins=9):
        self.nbins = nbins

    def _calc_gradient_discret(self, img):
        n_x, n_y = img.shape
        histogram = numpy.zeros((4, 4, self.nbins))

        for i in range(0, n_x) :
            for j in range(0, n_y) :
                dx = 0
                dy = 0
                if i < n_x - 1 :
                    dx += img[i + 1, j]
                if i > 0 :
                    dx -= img[i - 1, j]
                if j < n_y - 1 :
                    dy += img[i, j + 1]
                if j > 0 :
                    dy -= img[i, j - 1]

                if dy == 0 and dx == 0 :
                    continue

                magnitude = numpy.sqrt(dx**2 + dy**2)
                if dx == 0 :
                    angle = numpy.pi / 2
                else:
                    angle = numpy.arctan(dy / dx)
                    angle = (angle + numpy.pi / 2) / (numpy.pi / self.nbins)
               
                bin_pos = int(numpy.floor(angle))

                if bin_pos == self.nbins :
                    bin_pos = 0
                    angle = 0

                closest_bin = bin_pos

                if bin_pos == 0 :
                    if angle < 0.5 :
                        second_closest_bin = self.nbins - 1
                    else:
                        second_closest_bin = 1
                elif bin_pos == self.nbins - 1 :
                    if angle < self.nbins - 0.5 :
                        second_closest_bin = self.nbins - 2
                    else:
                        second_closest_bin = 0
                else:
                    if angle < bin_pos + 0.5 :
                        second_closest_bin = bin_pos - 1
                    else:
                        second_closest_bin = bin_pos + 1

                if angle < bin_pos + 0.5 :
                    second_closest_bin_distance = angle - (bin_pos - 0.5)
                else:
                    second_closest_bin_distance = (bin_pos + 1.5) - angle

                r = second_closest_bin_distance
                histogram[i / 8, j / 8, closest_bin] += r * magnitude
                histogram[i / 8, j / 8, second_closest_bin] += (1 - r) * magnitude

        vec = numpy.zeros((3, 3, self.nbins * 4))

        for i in range(3):
            for j in range(3):
                aux = histogram[i:i + 2, j:j + 2, :].flatten().copy()
                aux = aux / numpy.linalg.norm(aux)
                vec[i, j, :] = aux

        return vec.flatten()

    
    def _calc_gradient_one_image(self, img):
        nchannels = img.shape[2]
        vec = []

        for i in range(nchannels):
            vec.append(self._calc_gradient_discret(img[:,:,i]))

        return numpy.array(vec).flatten()

    
    def hog(self, X):
        n = X.shape[0]
        X_new = []

        for i in range(n):
            X_new.append(self._calc_gradient_one_image(X[i,:,:,:]))

        return numpy.array(X_new)

# Tests
Afin de tester nos modèles sur nos données d'entraînement, on va découper notre base d'entraînement en train/test à hauteur de 60/40%.

Pour plus de clarté, on ne présente ici que les modèles optimaux pour chacune des étapes suivantes (avant HOG, après ACP, après HOG, après HOG+ACP), i.e. la méthode optimale avec les paramètres optimaux parmi un nombre fini de modèles (les 2 paramètres variant de 1 à 10). Pour s'en assurer, il suffit de faire varier les paramètres à chaque fois (par une simple boucle sur ces paramètres).

### Avant HOG : SVM One Vs Rest (C=6,sigma=2) 

In [11]:
X_train_1 = df_train[0:3000]
X_test_1 = df_train[3000:5000]
Y_train_1 = y_train[0:3000]
Y_test_1 = y_train[3000:5000]
print(X_train_1.shape)
print(X_test_1.shape)
Y_train_1["Prediction"].value_counts()

(3000, 3072)
(2000, 3072)


3    311
1    308
0    306
6    305
8    304
4    298
5    294
7    292
9    291
2    291
Name: Prediction, dtype: int64

In [12]:
y_predict_OvR=SVMOvR(X_train_1,X_test_1,Y_train_1,6,2)

Classe : 0
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -1.1141e+03 -5.4312e+04  1e+05  4e-01  2e-14
 1: -9.3079e+02 -1.0934e+04  1e+04  2e-16  2e-14
 2: -1.2907e+03 -4.2159e+03  3e+03  2e-16  2e-14
 3: -1.4542e+03 -2.4599e+03  1e+03  1e-16  3e-14
 4: -1.5245e+03 -1.7928e+03  3e+02  2e-16  3e-14
 5: -1.5499e+03 -1.6173e+03  7e+01  2e-16  3e-14
 6: -1.5575e+03 -1.5642e+03  7e+00  2e-16  3e-14
 7: -1.5588e+03 -1.5591e+03  4e-01  2e-16  3e-14
 8: -1.5588e+03 -1.5589e+03  1e-02  2e-16  3e-14
 9: -1.5589e+03 -1.5589e+03  4e-04  2e-16  3e-14
Optimal solution found.
Nombre de vecteurs supports : 3000
Biais: 0.00733333333333
Temps pour l'entrainement : 58.527278
Temps pour la prédiction : 2.6031779999999998
Classe : 1
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -5.1956e+02 -5.6036e+04  1e+05  5e-01  2e-14
 1: -1.4963e+02 -1.1295e+04  1e+04  2e-16  3e-14
 2: -6.3763e+02 -3.6711e+03  3e+03  2e-

In [13]:
compt=Y_test_1.values[:,0]-y_predict_OvR
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_1))

Pourcentage de bonnes prédictions : 0.207


### Après ACP : SVM One Vs Rest (C=6,sigma=2, 30 composantes conservées) 

In [ ]:
X_train_kacp = pandas.DataFrame(kpca(df_train,2,30))[0:3000]
X_test_kacp = pandas.DataFrame(kpca(df_train,2,30))[3000:5000]
Y_train_1 = y_train[0:3000]
Y_test_1 = y_train[3000:5000]
print(X_train_1.shape)
print(X_test_1.shape)
Y_train_1["Prediction"].value_counts()

(3000, 3072)
(2000, 3072)


3    311
1    308
0    306
6    305
8    304
4    298
5    294
7    292
9    291
2    291
Name: Prediction, dtype: int64

In [ ]:
y_predict_OvR=SVMOvR(X_train_kacp,X_test_kacp,Y_train_1,6,2)

Classe : 0
Taille de la matrice de Gram : (3000, 3000)
     pcost       dcost       gap    pres   dres
 0: -3.8069e+03 -4.7301e+04  8e+04  3e-01  7e-14
 1: -3.5143e+03 -1.0779e+04  7e+03  2e-16  9e-14
 2: -3.5976e+03 -4.0723e+03  5e+02  2e-16  8e-14
 3: -3.6332e+03 -3.9621e+03  3e+02  1e-16  7e-14
 4: -3.6558e+03 -3.8455e+03  2e+02  2e-16  7e-14
 5: -3.6708e+03 -3.6920e+03  2e+01  2e-16  8e-14
 6: -3.6720e+03 -3.6779e+03  6e+00  1e-16  7e-14
 7: -3.6721e+03 -3.6775e+03  5e+00  2e-16  7e-14
 8: -3.6723e+03 -3.6763e+03  4e+00  2e-16  7e-14
 9: -3.6725e+03 -3.6756e+03  3e+00  2e-16  7e-14
10: -3.6726e+03 -3.6750e+03  2e+00  2e-16  7e-14
11: -3.6728e+03 -3.6746e+03  2e+00  2e-16  8e-14
12: -3.6729e+03 -3.6743e+03  1e+00  2e-16  9e-14
13: -3.6730e+03 -3.6740e+03  1e+00  2e-16  9e-14
14: -3.6731e+03 -3.6737e+03  6e-01  2e-16  1e-13
15: -3.6732e+03 -3.6735e+03  4e-01  2e-16  1e-13
16: -3.6732e+03 -3.6734e+03  2e-01  2e-16  1e-13
17: -3.6733e+03 -3.6733e+03  8e-02  2e-16  1e-13


In [ ]:
compt=Y_test_1.values[:,0]-y_predict_OvR
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_1))

### Après HOG :  SVM One Vs Rest (C=6, sigma=2) 

In [ ]:
# Création de la nouvelle base X obtenue après extraction de features

Xtrain = numpy.reshape(X_train, (X_train.shape[0], 3, 32, 32))
Xtrain = numpy.swapaxes(Xtrain, 1, 2)
Xtrain = numpy.swapaxes(Xtrain, 2, 3)

Xtest = numpy.reshape(X_test, (X_test.shape[0], 3, 32, 32))
Xtest = numpy.swapaxes(Xtest, 1, 2)
Xtest = numpy.swapaxes(Xtest, 2, 3)

histogram=HOG(nbins=9)
h=histogram.hog(Xtrain)
g=histogram.hog(Xtest)

X_tr = pandas.DataFrame(h)
X_te = pandas.DataFrame(g)

In [ ]:
X_train_2 = X_tr[0:3000]
X_test_2 = X_tr[3000:5000]
Y_train_2 = y_train[0:3000]
Y_test_2 = y_train[3000:5000]
print(X_train_2.shape)
print(X_test_2.shape)
Y_train_2["Prediction"].value_counts()

In [ ]:
y_predict_OvR=SVMOvR(X_train_2,X_test_2,Y_train_2,6,2)

In [ ]:
compt=Y_test_2.values[:,0]-y_predict_OvR
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_2))

### Après HOG + ACP :  SVM One Vs Rest (C=6, sigma=2, 30 composantes conservées) 

In [ ]:
X_train_3 = pandas.DataFrame(kpca(X_tr,2,30))[0:3000]
X_test_3 = pandas.DataFrame(kpca(X_tr,2,30))[3000:5000]
Y_train_3 = y_train[0:3000]
Y_test_3 = y_train[3000:5000]
print(X_train_3.shape)
print(X_test_3.shape)
Y_train_3["Prediction"].value_counts()

In [ ]:
y_predict_OvR=SVMOvR(X_train_3,X_test_3,Y_train_3,6,2)

In [ ]:
compt=Y_test_3.values[:,0]-y_predict_OvR
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_3))

# On soumet nos résultats finaux sur le site Kaggle

### Le modèle optimal est un SVM One Vs Rest (paramètres C=6 et sigma=2) après HOG mais sans ACP

In [31]:
final_pred=SVMOvR(X_tr,X_te,y_train,6,2)

Classe : 0
Taille de la matrice de Gram : (5000, 5000)


KeyboardInterrupt: 

In [ ]:
results = [int(i) for i in final_pred]
X_result = pandas.DataFrame(results,columns=['Prediction'])
X_result.index += 1
X_result.to_csv('Yte.csv',index=True,index_label='Id')